#### Author : Jeonghun Yoon

Naive Bayes classifier를 이용하여 영화 리뷰를 예측하는 감정 분류기를 구현하라. 
 - 0 : 부정
 - 1 : 긍정

In [174]:
import pandas as pd

In [2]:
# 영화 리뷰를 load한다. 사랑/장르라는 단어를 포함하고 있는 document를 load 한다.
reviews = pd.read_csv('./inputs/ratings_train.txt', delimiter='\t')

In [3]:
# 데이터 확인
reviews.head(10)

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1
5,5403919,막 걸음마 뗀 3세부터 초등학교 1학년생인 8살용영화.ㅋㅋㅋ...별반개도 아까움.,0
6,7797314,원작의 긴장감을 제대로 살려내지못했다.,0
7,9443947,별 반개도 아깝다 욕나온다 이응경 길용우 연기생활이몇년인지..정말 발로해도 그것보단...,0
8,7156791,액션이 없는데도 재미 있는 몇안되는 영화,1
9,5912145,왜케 평점이 낮은건데? 꽤 볼만한데.. 헐리우드식 화려함에만 너무 길들여져 있나?,1


In [219]:
neg = reviews[(reviews.document.str.len() >= 30) & (reviews.label == 0)].sample(3000, random_state=43)
pos = reviews[(reviews.document.str.len() >= 30) & (reviews.label == 1)].sample(3000, random_state=43)

In [220]:
# 형태소 분석기
import re
import konlpy
from konlpy.tag import Okt

okt = Okt()

In [221]:
def parse(s):
    s = re.sub(r'[?$.!,-_\'\"(){}~]+', '', s)
    try:
        return okt.nouns(s)
    except:
        return []

In [222]:
neg['parsed_doc'] = neg.document.apply(parse)
pos['parsed_doc'] = pos.document.apply(parse)

In [223]:
neg.head()

,id,document,label,parsed_doc
149713,6674102,정말 댓글에 속아서 보게된 시간 아까운 영화. 전체적으로 코믹적인 요소를 많이 넣으...,0,"[정말, 댓글, 속, 시간, 영화, 전체, 코믹, 요소, 정말, 웃기, 스토리]"
52541,8514375,점수를 줘야 하는거임? 30년후 구구절절 설명하며 끝나는모습 이라니...,0,"[점수, 임, 년후, 절절, 설명, 모습]"
72078,1173600,주제는 좋으나 영화 자체는 주제를 포함하기엔 좀 조잡하다,0,"[주제, 영화, 자체, 주제, 포함, 좀]"
19403,8384839,"이걸 영화라고 만들다니...말그대로 '돈만 떡으로 바르고 여운의 감동, 의미는 없는...",0,"[걸, 영화, 그대로, 돈, 떡, 여운, 감동, 의미, 영화, 이건, 그냥, 영화,..."
106111,8997490,"과장되고 어설픈 연기에 인류의 보편적 정서와는 거리가 먼 일본식 정서간의 괴리감, ...",0,"[과장, 연기, 인류, 보편, 정서, 거리, 일본, 정서, 괴리감, 무엇, 공포영화..."


In [224]:
pos.head()

,id,document,label,parsed_doc
148005,8918849,마지막에 눈물이 흐를뻔... 너무 감동적이고 재미있는 영화,1,"[마지막, 눈물, 뻔, 감동, 영화]"
41010,9304354,오! 평점 높네요. 저도 진짜 좋아하는 영화에요. 소소한 일상의 반전. 아즈키판다 짱!,1,"[오, 평점, 저, 진짜, 영화, 일상, 반전, 키, 판다, 짱]"
97924,9324301,전 아프리카 구호 활동중 에볼라에 걸려 눈을 감는 순간 누군가 틀어준 이 영화를 보...,1,"[전, 아프리카, 구호, 활동, 에볼라, 눈, 순간, 누군가, 이, 영화, 보고, ..."
68271,10274621,전작을 본적이 없는데도 너무 재미있게 봤습니다. 크리스 에반스와 사무엘 L.잭슨의 ...,1,"[전작, 본적, 크리스, 에반스, 사무엘, 잭슨, 카리스마, 그, 외, 스토리, 액..."
41106,9908746,신선하고 환경파괴에대해서 자연과 인간의 공존 전쟁 등등 많은 교훈을 주는 애니메이션...,1,"[환경, 파괴, 대해, 자연, 인간, 공존, 전쟁, 등등, 교훈, 애니메이션, 원작..."


In [244]:
neg_train = neg[:2900]
pos_train = pos[:2900]
neg_test = neg[2900:]
pos_test = pos[2900:]

Corpus 생성하기

In [245]:
train_data = pd.concat([neg_train, pos_train], axis=0)
train_data.head(1)

,id,document,label,parsed_doc
149713,6674102,정말 댓글에 속아서 보게된 시간 아까운 영화. 전체적으로 코믹적인 요소를 많이 넣으...,0,"[정말, 댓글, 속, 시간, 영화, 전체, 코믹, 요소, 정말, 웃기, 스토리]"


In [246]:
corpus = train_data.groupby('label').agg({'parsed_doc':sum})
corpus

,parsed_doc
label,
0,"[정말, 댓글, 속, 시간, 영화, 전체, 코믹, 요소, 정말, 웃기, 스토리, 점..."
1,"[마지막, 눈물, 뻔, 감동, 영화, 오, 평점, 저, 진짜, 영화, 일상, 반전,..."


In [247]:
from collections import Counter

In [248]:
neg_corpus = Counter(corpus.parsed_doc.iloc[0]).most_common()
neg_corpus[:10]

[('영화', 1376),
 ('진짜', 304),
 ('스토리', 242),
 ('점', 222),
 ('이', 211),
 ('것', 209),
 ('평점', 206),
 ('정말', 199),
 ('감독', 195),
 ('왜', 193)]

In [249]:
pos_corpus = Counter(corpus.parsed_doc.iloc[1]).most_common()
pos_corpus[:10]

[('영화', 1502),
 ('정말', 357),
 ('이', 275),
 ('것', 248),
 ('최고', 244),
 ('연기', 237),
 ('생각', 218),
 ('때', 204),
 ('진짜', 196),
 ('감동', 192)]

$$p(spam|doc) = \frac{p(doc|spam) \times p(spam)}{p(doc)} = \frac{\Pi_{i=1}^{n}p(word_i|spam) \times p(spam)}{p(doc)}$$

In [250]:
import numpy as np

`'tuple' object does not support item assignment``

In [251]:
neg_words, neg_cnts = zip(*neg_corpus)
neg_likelihoods = np.array(neg_cnts) / len(neg_train)
neg_likelihoods

array([4.74482759e-01, 1.04827586e-01, 8.34482759e-02, ...,
       3.44827586e-04, 3.44827586e-04, 3.44827586e-04])

In [252]:
pos_words, pos_cnts = zip(*pos_corpus)
pos_likelihoods = np.array(pos_cnts) / len(pos_train)
pos_likelihoods

array([5.17931034e-01, 1.23103448e-01, 9.48275862e-02, ...,
       3.44827586e-04, 3.44827586e-04, 3.44827586e-04])

In [253]:
neg_dicts = {}
for w, v in zip(neg_words, neg_likelihoods):
    neg_dicts[w] = v

In [254]:
neg_dicts

{'영화': 0.47448275862068967,
 '진짜': 0.10482758620689656,
 '스토리': 0.08344827586206896,
 '점': 0.07655172413793103,
 '이': 0.07275862068965518,
 '것': 0.07206896551724137,
 '평점': 0.07103448275862069,
 '정말': 0.06862068965517241,
 '감독': 0.06724137931034482,
 '왜': 0.06655172413793103,
 '사람': 0.06551724137931035,
 '말': 0.06413793103448276,
 '연기': 0.06310344827586208,
 '뭐': 0.061724137931034484,
 '시간': 0.05758620689655172,
 '배우': 0.05586206896551724,
 '좀': 0.05241379310344828,
 '쓰레기': 0.05241379310344828,
 '생각': 0.0503448275862069,
 '내용': 0.05,
 '그냥': 0.04931034482758621,
 '더': 0.04758620689655172,
 '드라마': 0.04586206896551724,
 '거': 0.04482758620689655,
 '이건': 0.04379310344827586,
 '재미': 0.04310344827586207,
 '돈': 0.04068965517241379,
 '보고': 0.04,
 '때': 0.039655172413793106,
 '내': 0.039310344827586205,
 '하나': 0.03896551724137931,
 '걸': 0.037241379310344824,
 '그': 0.035517241379310345,
 '최악': 0.035517241379310345,
 '주인공': 0.03310344827586207,
 '정도': 0.03206896551724138,
 '별로': 0.030689655172413795

In [255]:
pos_dicts = {}
for w, v in zip(pos_words, pos_likelihoods):
    pos_dicts[w] = v

In [256]:
pos_dicts

{'영화': 0.5179310344827586,
 '정말': 0.12310344827586207,
 '이': 0.09482758620689655,
 '것': 0.08551724137931034,
 '최고': 0.08413793103448276,
 '연기': 0.08172413793103449,
 '생각': 0.07517241379310345,
 '때': 0.0703448275862069,
 '진짜': 0.06758620689655172,
 '감동': 0.06620689655172414,
 '드라마': 0.0596551724137931,
 '사람': 0.05827586206896552,
 '보고': 0.05758620689655172,
 '더': 0.05655172413793103,
 '점': 0.05482758620689655,
 '수': 0.05241379310344828,
 '배우': 0.05241379310344828,
 '그': 0.05103448275862069,
 '평점': 0.04896551724137931,
 '스토리': 0.04862068965517241,
 '마지막': 0.047241379310344826,
 '사랑': 0.04586206896551724,
 '다시': 0.04379310344827586,
 '내': 0.043448275862068966,
 '지금': 0.04103448275862069,
 '말': 0.04,
 '작품': 0.039310344827586205,
 '왜': 0.033448275862068964,
 '이야기': 0.033448275862068964,
 '나': 0.03310344827586207,
 '장면': 0.031379310344827584,
 '재미': 0.031379310344827584,
 '내용': 0.029310344827586206,
 '볼': 0.02793103448275862,
 '거': 0.02793103448275862,
 '처음': 0.027586206896551724,
 '정도': 0.0

# Test

In [257]:
test_data = pd.concat([neg_test, pos_test], axis=0)

In [258]:
def predict(doc):
    pos_prior, neg_prior = 1/2, 1/2

    # Posterior of pos
    pos_prob = 1
    for word in doc:
        if word in pos_dicts:
            pos_prob *= (pos_dicts[word] * len(pos_train) + 1) / (len(pos_train) + len(pos_dicts))
        else:
            pos_prob *= 1 / len(pos_dicts)
    pos_prob *= pos_prior
    
    # Posterior of neg
    neg_prob = 1
    for word in doc:
        if word in neg_dicts:
            neg_prob *= (neg_dicts[word] * len(neg_train) + 1) / (len(neg_train) + len(neg_dicts))
        else:
            neg_prob *= 1 / len(neg_dicts)
    neg_prob *= neg_prior
    
    print(pos_prob)
    print(neg_prob)
    
    if pos_prob >= neg_prob:
        return 1
    else:
        return 0

In [259]:
test_data['pred'] = test_data.parsed_doc.apply(predict)

1.4361124779083047e-12
1.3634077047093028e-12
2.2926510411671835e-20
2.6109375305809047e-18
3.902858368359957e-51
1.2912351073605147e-50
2.490075930926413e-13
2.593680194473828e-13
2.5460264677497373e-64
1.6557982323726164e-67
3.878941984946904e-74
4.3473959731514576e-70
2.3896947449609412e-11
1.9849975456263203e-11
9.285100476397855e-38
6.7782174987055535e-34
5.707215957020525e-37
5.317595950130026e-37
5.119785448213085e-36
6.221620950101689e-35
8.152559980110954e-19
4.735438854695832e-18
8.050184624155021e-24
4.350950816707652e-22
1.8581335762822078e-42
1.7591132411382753e-39
1.3194046930761191e-25
3.891638265266564e-22
4.728808046195524e-17
1.6509188382703995e-16
2.994625293974937e-19
8.05394804861529e-19
1.0189252347432296e-28
2.1987063186778415e-27
2.280059747420381e-30
1.0877132424643742e-29
1.5416817089440565e-12
1.906771041365185e-11
2.5127161254469743e-51
1.747184727849597e-47
3.218511600378482e-11
8.302093198664707e-11
6.59213219354629e-11
2.7879760269508554e-11
1.17141157565

In [260]:
test_data

,id,document,label,parsed_doc,pred
91417,5079332,쉴새없이 떠드는 폴도 싫지만...영화를 이렇게 만든 감독은 더 싫다.,0,"[새, 폴, 영화, 감독, 더]",1
122641,2776735,별표 반개도 아깝다..2시간 동안 본 짜증이 확밀려오네..,0,"[별표, 반개, 시간, 동안, 짜증, 확]",0
72606,8477996,내용은 봐줄만해. 하지만 15금과 19금 사이에 서있다. 포스터는 호러 귀신나올것같...,0,"[내용, 만해, 금, 금, 사이, 포스터, 호러, 귀신, 막상, 차마, 두, 최악,...",0
138243,7923759,진정한 수면제...졸다가 비명지르는 소리에 깼다가 다시 자다,0,"[수면제, 비명, 소리, 다시]",0
118247,8142893,"""진짜 슈스케 버스커 버스커, 허각, 울랄라세션 나올땐 재밌는데 갈수록 재미가 없어...",0,"[진짜, 슈스케, 버스커, 버스커, 허각, 울랄라세션, 땐, 갈수록, 재미, 제, ...",1
63089,8525344,내가씨x 아침에 티비에서 틀어주는거 보고 욕이 나와서 평점쓴다. 개그맨 성우새.끼들...,0,"[가씨, 아침, 티비, 보고, 욕, 평점, 개그맨, 성우, 새끼, 조화, 는걸, 영...",0
132881,1872310,이송희일 : 관심받고싶어서 디워까댄거에요 죄송해요 ㅠㅠ,0,"[이송희일, 관심, 디워]",1
140268,4712973,1등!! 이건 무슨 병맛 출연진에 뭐냐 이거 긴급조치 19호 스맬난다...,0,"[등, 이건, 무슨, 병맛, 출연, 진, 뭐, 거, 긴급, 조치, 호, 스맬난다]",0
26536,5799940,"모모는 철부지,모모는 무지개,모모는 생을 쫒아가는 시계바늘이다. 이노래모르면?간첩",0,"[모모, 철부지, 모모, 무지개, 모모, 생, 시계, 바늘, 노래, 간첩]",1
9821,8951429,진짜한편봤는데. 정말 다들 무례하네요.음식의 평가를 제대로하는게아니라 깎아내리기 바...,0,"[진짜, 한편, 정말, 음식, 평가, 제대로, 리기, 프로그램, 취지, 황은정, 최...",0


In [261]:
sum(test_data.label ^ test_data.pred)

46

In [262]:
46/200

0.23

# TF-IDF